In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
all_data = [train_data, test_data]
#print(all_data)

In [3]:
#printing top 5 data 
train_data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
#print top 5 test data 
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
#feature eng.
#Pclass
#servied rate on pclass

train_data[['Pclass','Survived']].groupby(['Pclass']).mean()

,Survived
Pclass,
1,0.629630
2,0.472826
3,0.242363


In [6]:
#servied rate on sex
train_data[['Sex', 'Survived']].groupby(['Sex']).mean()

,Survived
Sex,
female,0.742038
male,0.188908


In [7]:
print(train_data['SibSp'].dtype)
print(train_data['Parch'].dtype)

int64
int64


In [8]:
#family size also matters
for data in all_data:
    data['family_size'] = data['SibSp'] + data['Parch'] + 1
print( train_data[["family_size","Survived"]].groupby(["family_size"], as_index = False).mean() )

   family_size  Survived
0            1  0.303538
1            2  0.552795
2            3  0.578431
3            4  0.724138
4            5  0.200000
5            6  0.136364
6            7  0.333333
7            8  0.000000
8           11  0.000000


In [9]:
# Feature :embarked
for data in all_data:
    data['Embarked'] = data['Embarked'].fillna('S')
    
train_data[['Embarked','Survived']].groupby(['Embarked']).mean()
    

,Survived
Embarked,
C,0.553571
Q,0.389610
S,0.339009


In [10]:
#Feature Fare;
for data in all_data:
    data['Fare'] = data['Fare'].fillna(data['Fare'].median())
    
train_data['category_fare'] = pd.qcut(train_data['Fare'], 4)
train_data[['category_fare','Survived']].groupby('category_fare').mean()

,Survived
category_fare,
"(-0.001, 7.91]",0.197309
"(7.91, 14.454]",0.303571
"(14.454, 31.0]",0.454955
"(31.0, 512.329]",0.581081


In [11]:
#feature age:
for data in all_data:
    age_avg = data['Age'].mean()
    age_std = data['Age'].std()
    age_null = data['Age'].isnull().sum()
    random_list = np.random.randint(age_avg - age_std, age_avg + age_std , size = age_null)
    data['Age'][np.isnan(data['Age'])] = random_list
    data['Age'] = data['Age'].astype(int)

train_data['category_age'] = pd.cut(train_data['Age'], 5)
    
train_data[['category_age','Survived']].groupby([train_data['category_age']]).mean()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Survived
category_age,
"(-0.08, 16.0]",0.508929
"(16.0, 32.0]",0.361556
"(32.0, 48.0]",0.366412
"(48.0, 64.0]",0.434783
"(64.0, 80.0]",0.090909


In [12]:
#feature title  
#getting title from name using python regex
import re
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\. ', name)
    if title_search:
        return title_search.group(1)
    return " "

for data in all_data:
    data['title'] = data['Name'].apply(get_title)
    
for data in all_data:
    data['title'] = data['title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'],'Rare')
    data['title'] = data['title'].replace('Mlle','Miss')
    data['title'] = data['title'].replace('Ms','Miss')
    data['title'] = data['title'].replace('Mme','Mrs')

#print cross tab 
#print(pd.crosstab(train_data['title'],train_data['Sex']))

train_data[['title', 'Survived']].groupby(['title']).mean()


,Survived
title,
Master,0.575000
Miss,0.702703
Mr,0.156673
Mrs,0.793651
Rare,0.347826


In [13]:
sex_map = { 'female':0 , 'male':1 }
embark_map = {'S':0, 'C':1, 'Q':2}
title_map = {'Mr':1, 'Miss':2, 'Mrs':3, 'Master':4, 'Rare':5}

for data in all_data:

    #Mapping Sex
    data['Sex'] = data['Sex'].map(sex_map).astype(int)
    
    #Mapping Title
    data['title'] = data['title'].map(title_map)
    data['title'] = data['title'].fillna(0)
    
    #Mapping Embarked
    data['Embarked'] = data['Embarked'].map(embark_map).astype(int)
      #Mapping Fare
    data.loc[ data['Fare'] <= 7.91, 'Fare']                            = 0
    data.loc[(data['Fare'] > 7.91) & (data['Fare'] <= 14.454), 'Fare'] = 1
    data.loc[(data['Fare'] > 14.454) & (data['Fare'] <= 31), 'Fare']   = 2
    data.loc[ data['Fare'] > 31, 'Fare']                               = 3
    data['Fare'] = data['Fare'].astype(int)
    
     #Mapping Age
    data.loc[ data['Age'] <= 16, 'Age']                       = 0
    data.loc[(data['Age'] > 16) & (data['Age'] <= 32), 'Age'] = 1
    data.loc[(data['Age'] > 32) & (data['Age'] <= 48), 'Age'] = 2
    data.loc[(data['Age'] > 48) & (data['Age'] <= 64), 'Age'] = 3
    data.loc[ data['Age'] > 64, 'Age']  
    
    #Feature Selection
#Create list of columns to drop
drop_elements = ["Name", "Ticket", "Cabin", "SibSp", "Parch", "family_size"]

#Drop columns from both data sets
train_data = train_data.drop(drop_elements, axis = 1)
train_data = train_data.drop(['PassengerId','category_fare', 'category_age'], axis = 1)
test_data = test_data.drop(drop_elements, axis = 1)

    
train_data.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,title
0,0,3,1,1,0,0,1
1,1,1,0,2,3,1,3
2,1,3,0,1,1,0,2
3,1,1,0,2,3,0,3
4,0,3,1,2,1,0,1


In [14]:
#Prediction
#Train and Test data
X_train = train_data.drop("Survived", axis=1)
Y_train = train_data["Survived"]
X_test  = test_data.drop("PassengerId", axis=1).copy()
#print(X_train)
#print(Y_train)

In [15]:
# #prediction
# from sklearn.tree import DecisionTreeClassifier
# decision_tree = DecisionTreeClassifier()
# decision_tree.fit(X_train, Y_train)

# y_pred = decision_tree.predict(X_test)
# accuracy = round(decision_tree.score(X_train, Y_train) * 100, 2)
# print("Model Accuracy: ",accuracy)
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=50, random_state=0)
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
accuracy = round(clf.score(X_train, Y_train) * 100, 2)
print("Model Accuracy: ",accuracy)



Model Accuracy:  86.08


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [16]:
#knn lagorithm
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier(n_neighbors = 18)
knn.fit(X_train, Y_train)
ypred1 = knn.predict(X_test)
knn_accuracy = round(knn.score(X_train, Y_train) * 100, 2)
print("Model Accuracy: ",knn_accuracy)

Model Accuracy:  82.49


In [17]:
#logistic
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=0)
lr.fit(X_train, Y_train)
ypred2 = lr.predict(X_test)
knn_accuracy1 = round(lr.score(X_train, Y_train) * 100, 2)
print("Model Accuracy: ",knn_accuracy1)


Model Accuracy:  78.11


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [18]:
from sklearn.ensemble import GradientBoostingClassifier
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=40, learning_rate=learning_rate, max_features=2, max_depth=10, random_state=0)
    gb_clf.fit(X_train, Y_train)
    gb_pred = gb_clf.predict(X_test)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, Y_train)))
   # print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_val, y_val)))

Learning rate:  0.05
Accuracy score (training): 0.869
Learning rate:  0.075
Accuracy score (training): 0.869
Learning rate:  0.1
Accuracy score (training): 0.869
Learning rate:  0.25
Accuracy score (training): 0.869
Learning rate:  0.5
Accuracy score (training): 0.869
Learning rate:  0.75
Accuracy score (training): 0.869
Learning rate:  1
Accuracy score (training): 0.869


In [19]:
#final csv_creation
submission = pd.DataFrame({
    'PassengerId':test_data['PassengerId'],
    "Survived": gb_pred  
})

submission.to_csv('submission_1.csv', index = False)